# Convolutional Neural Network 실습

## 손으로 쓴 숫자 인식 (Handwritten Digit Recognition)

손으로 쓴 숫자를 모아 놓은 [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset을 학습해서 손으로 쓴 숫자가 어떤 숫자인지를 분류하는 deep learning 모델을 작성하는 법을 배웁니다. 이 예제는 Deep Learning에서는 "Hello World" 같은 것입니다.

> MNIST는 손으로 쓴 숫자를 모아 놓은 데이터입니다. 28x28 픽셀의 회식 이미지를 70,000개의 숫자가 10개의 클래스로 구분되어 있습니다.
> ![png](https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/example/mnist.png)
>
> **Figure 1:** Sample images from the MNIST dataset.

이 예제에서는 60,000개를 학습 이미지로 사용하고, 10,000개를 테스트 이미지로 사용합니다. 이 실습에서는 아래 두가지 방법으로 숫자 분류 모델을 만들어 보겠습니다.

* Multilayer Perceptron (MLP)
* Convolutional Neural Network (CNN)

## 데이터 로딩

모델을 정의하기 전에 [MNIST](http://yann.lecun.com/exdb/mnist/) dataset을 로딩합니다. 아래 코드는 이미지 데이터셋을 다운로드해서, 이미지와 레벨을 메모리에 저장합니다.

In [ ]:
import mxnet as mx
import matplotlib.pyplot as plt

mnist = mx.test_utils.get_mnist()

위 코드를 수행하면, MNIST 데이터셋 전체가 메모리로 올라갑니다. 만약, 데이터셋이 더 큰 경우에는 모든 데이터셋을 메모리에 미리 올리는 것인 불가능할 것입니다. 이를 해결하기 위해서는 데이터 소스로부터 빠르고 효과적으로 데이터를 스트리밍할 수 있는 방법이 필요한데, MXNet Data iterator들이 이런 기능을 제공합니다. 이 iterator들은 초기화가 쉽고, 속도 최적화가 되어 있습니다. 학습이 수행되는 동안, 학습 데이터는 작은 배치로 나뉘어서 사용되고, 각 학습 샘플들은 전체 학습이 종료될 때까지 몇차례 재사용됩니다.

이 실습에서는 data iterator를 한 배치에 100개의 데이터에 사용되도록 설정합니다. 각 샘플은 28x28 픽셀의 회색 이미지와 어떤 숫자인지를 알려주는 라벨로 구성되어 있습니다.

이미지 배치는 보통 4차원 행렬로 표현됩니다. 이 행렬의 shape는 `(batch_size, num_channels, width, height)` 입니다. MNIST 이미지가 회색이기 때문에, 색깔 채널은 한개입니다. 즉, 입력의 shape는 `(batch_size, 1, 28, 28)`이 됩니다.

학습 데이터를 사용하는데 중요한 점은, 라벨 순서대로 나열된 데이터를 그대로 사용하지 말아야된다는 것입니다. 만약 같은 라벨의 데이터를 묶어서 학습한다면 학습 속도가 느려집니다. Data iterator는 입력 데이터를 자동으로 섞어줍니다. 테스트 데이터는 섞을 필요 없습니다.

아래 코드는 data iterator들은 초기화해서, 학습 데이터와 테스트 데이터를 주는 iterator를 생성합니다.

In [ ]:
batch_size = 100
train_iter = mx.io.NDArrayIter(mnist['train_data'], mnist['train_label'], batch_size, shuffle=True)
val_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)

## 학습

MNIST 데이터를 학습해서 모델은 만드는 다양한 방법 중에, 

(1) 전통적인 deep neural network 아키텍처, Multilayer Percepton (MLP),

(2) Convolutional Neural Network 아키텍처

를 어떻게 구성하는지 알아보겠습니다. 

### 모델 1 - [Multilayer Perceptron](https://en.wikipedia.org/wiki/Multilayer_perceptron) 

MXNet의 Symbol API를 이용해서 MLP를 정의합니다. 우선 입력 데이터를 저장할 place holder를 생성합니다. MLP를 다룰때는, 28x28 이미지를 784 ( 28 * 28 ) 크기의 1차원 행렬로 변환해서 사용해야 합니다. 1차원으로 펼칠 때 원소들의 순서는 모든 이미지에 동일하게 적용된다면 펼치는 순서는 중요하지 않습니다.

### 데이터 pre-prosessing

In [ ]:
data = mx.sym.var('data')
# Flatten the data from 4-D shape into 2-D (batch_size, num_channel*width*height)
data = mx.sym.flatten(data=data)

다차원 배열을 flat하게 만들면서 중요한 정보가 손실되는 것이 아닐까요? 사실 그렇습니다. 이에 대해서는 입력 데이터의 모양을 보존하는 Convolutional Neural Network을 만들때 살펴보겠습니다. 우선은 flat한 이미지를 이용하겠습니다.

MLP는 몇개의 fully connected layer들로 구성됩니다. Fully connected layer (FC layer)는 각 neuron이 이전 layer의 모든 neuron들과 연결되어 있습니다. 선형대수 관점에서 보면, 하나의 FC layer는 *n x m* 입력 행렬 *X*에 [affine transform](https://en.wikipedia.org/wiki/Affine_transformation)을 적용해서, *n x k* 행렬 *Y*를 얻는 것을 의미합니다. 여기서 *k*는 FC layer의 neuron 개수를 의미합니다. FC layer는 아래 두개의 학습 파라메터를 갖습니다.

* *m x k* weight 행렬 *W*
* *m x 1* bias 백터 *b*

##### Activation 함수의 중요성
MLP에서 대부분의 FC layer는 activation 함수 연결됩니다. Activation 함수를 적용함으로 element-wise non-linearity를 갖게됩니다. 만약 없다면, 여러 FC layer들은 하나의 FC layer로 동작하게됩니다. 많이 사용되는 activation 함수는 [rectified linear unit](https://en.wikipedia.org/wiki/Rectifier_%28neural_networks%29) (ReLU), tanh 또는 sigmoid가 있습니다. 대부분의 경우 좋은 성능을 보여주는 ReLu를 많이 사용합니다.

### MLP 네트워크 정의

#### Fully Connected (128) --> ReLu --> Fully Connected (64) --> ReLu

다음 코드는 128개와 64개의 neuron을 갖는 Fully Connected layer 두개를 정의합니다. 이 두 FC layer 사이에는 ReLu activation를 적용합니다.

##### <font color='red'>문제</font>
* 첫번째 레이어에 activation 함수를 ReLu (relu)로 설정
* Neuron 개수가 다른 두번쨰 Fully-connected layer (activation 함수 포함) 정의

In [ ]:
# The first fully-connected layer and the corresponding activation function
fc1  = mx.sym.FullyConnected(data=data, num_hidden=128)
act1 = mx.sym.Activation(data=fc1, act_type="relu")

# The second fully-connected layer and the corresponding activation function
fc2  = ___ 두번째 레이어의 Fully Connected layer 정의 ___
act2 = ___ 두번째 레이어의 Activation 함수 적용 ___

마지막 Fully Connected layer는 결과 클래스 개수와 같은 수의 neuron을 갖도록 정의합니다. 이 layer에 대한 activation 함수는 softmax 함수를 사용합니다.

###### softmax 함수
softmax 함수는 입력값들을 각 클래스에 대한 확률값으로 변환합니다. 

###### loss 함수
loss 함수는 네트워크가 예측한 확률 분포 (예, softmax output)과 라벨의 실제 확률 분포 사이의 [cross entropy](https://en.wikipedia.org/wiki/Cross_entropy)를 계산합니다.

아래 코드는 마지막 크기가 10인 fully connected layer를 정의하고, 그 결과를  `SoftMaxOutput` layer로 연결합니다. 이 layer는 softmax과 cross-entropy loss를 동시에 계산합니다. loss는 학습 단계에서만 계산됩니다.


> <font color='red'>문제</font>
* 마지막 layer인 fc3에 몇개의 neuron이 필요한가요? 해당 숫자를 num_hidden 값으로 설정하세요

In [ ]:
# MNIST has 10 classes
fc3  = mx.sym.FullyConnected(data=act2, num_hidden=__Neuron 개수__)
# Softmax with cross entropy loss
mlp  = mx.sym.SoftmaxOutput(data=fc3, name='softmax')

![png](https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/image/mlp_mnist.png)

**Figure 2:** MLP network architecture for MNIST.

### 학습 수행

`module`의 high-level API를 이용해서 학습(training) 및 예측(inference)을 수행하는 코드를 작성합니다. `module` API는 학습을 어떻게 진행할 것인지 정하는 파라메터(hyper-parameter)를 설정할 수 있습니다.

* optimizer: SGD (Stochastic Gradient Descent)
* batch_size: 100
* learning_rate: 0.1
* epoch: 10

> **Standard SGD vs. mini-batch SGD**
> 
> Standard SGD : 하나의 데이터씩 학습을 수행함. 학습 속도가 느림
> mini-batch SGD : 전체 학습 데이터를 작은 집합 (batch)로 나눠서 mini-batch 단위로 학습을 수행함

> epoch: 학습 데이터 전체에 대한 수행 회수

학습은 최적의 파라메터(weight + bias)를 찾을 떄까지 학습 데이터에 대해서 반복적으로 수행합니다. 최적의 파라메터가 찾아졌는지 여부는 모델의 성능(정확도)가 수렴 여부를 보고 결정하게 됩니다.

In [ ]:
import logging
logging.getLogger().setLevel(logging.DEBUG)  # logging to stdout
# create a trainable module on CPU
mlp_model = mx.mod.Module(symbol=mlp, context=mx.cpu())
mlp_model.fit(train_iter,  # train data
              eval_data=val_iter,  # validation data
              optimizer='sgd',  # use SGD to train
              optimizer_params={'learning_rate':0.1},  # use fixed learning rate
              eval_metric='acc',  # report accuracy during training
              batch_end_callback = mx.callback.Speedometer(batch_size, 100), # output progress for each 100 data batches
              num_epoch=10)  # train for at most 10 dataset passes

### 예측 (Prediction)

학습이 완료되면, 테스트 데이터를 사용해서 예측을 수행하고 이를 기반으로 학습 모델을 평가합니다.

아래 코드는 각 테스트 이미지에 대한 예측 확률 점수(prediction probability score)를 계산합니다.

*prob[i][j]* : *i*-번째 테스트 이미지가 *j*-번째 결과 클래스일 확률

In [ ]:
test_iter = mx.io.NDArrayIter(mnist['test_data'], None, batch_size)
prob = mlp_model.predict(test_iter)
assert prob.shape == (10000, 10)

테스트 이미지에 대한 라벨 정보가 있기 때문에, accuracy metric을 다음과 같이 계산할 수 있습니다.

In [ ]:
test_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)
# predict accuracy of mlp
acc = mx.metric.Accuracy()
mlp_model.score(test_iter, acc)
print(acc)
assert acc.get()[1] > 0.96

잘 수행되었다만, accuracy 값이 0.96 정도로 나올것입니다. 이는, 숫자를 정확하게 맞출 확률이 96%라는 의미입니다. 다음 방법을 사용하면, 더 좋을 결과를 얻을 수 있습니다.

### Convolutional Neural Network

MLP를 이용해서 이미지를 분류하는 경우에는 이미지의 원래 모양을 고려하지 때문에, 가로축과 세로축을 따르는 공간적인 상관 관계에 대한 의미를 살려서 분석하지 못합니다. Convolutional Neural Network (CNN)은 이 문제를 더 구조화된 weight 구성을 통해서 해결합니다. 즉, 이미지를 1차원 배열로 변환해서 간단한 형렬 곱하기 연산을 수행하는 대신, 입력 이미지에 대한 2차원 convolution을 수행하는 여러개의 convolutional layer를 사용합니다.

#### Convolution Layer
하나의 convolution layer는 하나 이상의 filter를 갖습니다. 각 filter는 feature를 탐지하는 역할을 하는 filter들은 학습을 통해서 filter에 사용되는 적절한 파라메터를 찾아나가게 됩니다.

MLP와 유사하게 convolution layer의 결과는 non-linearity가 activation 함수를 통해서 적용되기도 합니다.

#### Pooling Layer
CNN의 또다른 중요한 레이어는 pooling layer입니다. A pooling layer serves to make the CNN translation invariant: a digit remains the same even when it is shifted left/right/up/down by a few pixels. pooling layer는 또한 *n x m* 패치를 단일 값으로 줄임으로 네트워크가 공간적인 위치에 덜 민감하도록 만드는 역할을 합니다.

Pooling layer는 늘 convolution (+activation) layer 다음에 위치합니다.

* conv layer --> activation layer --> pooling layer
* conv layer --> pooling layer

다음 코드는 LeNet이라는 Convolutional Neural Network 아키텍처를 정의합니다.

> LeNet: 숫자 분류에 대한 성능이 좋은 유명한 네트워크

> <font color='red'>문제</font>
> LeNet은 activation 함수로 sigmoid를 사용했지만, 여기서는 tanh activation 함수를 사용하겠습니다. act_type 값을 변경해보세요. (힌트: tanh )

In [ ]:
data = mx.sym.var('data')
# first conv layer
conv1 = mx.sym.Convolution(data=data, kernel=(5,5), num_filter=20)
tanh1 = mx.sym.Activation(data=conv1, act_type="sigmoid")
pool1 = mx.sym.Pooling(data=tanh1, pool_type="max", kernel=(2,2), stride=(2,2))
# second conv layer
conv2 = mx.sym.Convolution(data=pool1, kernel=(5,5), num_filter=50)
tanh2 = mx.sym.Activation(data=conv2, act_type="sigmoid")
pool2 = mx.sym.Pooling(data=tanh2, pool_type="max", kernel=(2,2), stride=(2,2))
# first fullc layer
flatten = mx.sym.flatten(data=pool2)
fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=500)
tanh3 = mx.sym.Activation(data=fc1, act_type="sigmoid")
# second fullc
fc2 = mx.sym.FullyConnected(data=tanh3, num_hidden=10)
# softmax loss
lenet = mx.sym.SoftmaxOutput(data=fc2, name='softmax')

![png](https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/image/conv_mnist.png)

**Figure 3:** First conv + pooling layer in LeNet.

위 MLP에서 사용한 hyper-parameter 값을 동일하게 LeNet에 적용하는 코드입니다. 

>  <font color='red'>문제</font>
아래 코드를 GPU를 사용하도록 변경해보세요. (힌트, mx.gpu())

In [ ]:
# create a trainable module on GPU 0

accu = []
def my_callback(param):
    _, value = param.eval_metric.get()
    accu.append(value)

lenet_model = mx.mod.Module(symbol=lenet, context=mx.cpu())
# train with the same
lenet_model.fit(train_iter,
                eval_data=val_iter,
                optimizer='sgd',
                optimizer_params={'learning_rate':0.1},
                eval_metric='acc',
                batch_end_callback = [mx.callback.Speedometer(batch_size, 100),my_callback],
                num_epoch=10)

In [ ]:
plt.plot(accu)
plt.ylabel('accuracy')
plt.xlabel('iterations (per hundreds)')
plt.title("Learning rate = 0.1")
plt.show()

### 예측 (Prediction)

학습이 완료된 LeNet 모델에 테스트 데이터를 이용해서 모델 성능을 평가합니다.

In [ ]:
test_iter = mx.io.NDArrayIter(mnist['test_data'], None, batch_size)
prob = lenet_model.predict(test_iter)
test_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)
# predict accuracy for lenet
acc = mx.metric.Accuracy()
lenet_model.score(test_iter, acc)
print(acc)
assert acc.get()[1] > 0.98

LeNet 모델의 성능(정확도)가 MLP의 성능보다 좋은 것을 볼 수 있습니다. 